In [499]:
from openai import OpenAI
from lmformatenforcer import JsonSchemaParser
from pydantic import BaseModel
import re, json, os
from typing import Optional, Type, TypeVar, List, Dict, Any
import datetime

In [2]:
# !which python

In [535]:
import os
from openai import AzureOpenAI

endpoint = "https://twgmm-m9bsqfti-eastus2.openai.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"

subscription_key = "9T33Ya79QaudQQABh7ZGjDL92cGHyFKLMIzwvchABHnYnCNDeLfgJQQJ99BDACHYHv6XJ3w3AAAAACOGjjRw"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

Paris is a city rich in history, culture, and art. Here are some must-see attractions and experiences you should consider during your visit:

### Iconic Landmarks
1. **Eiffel Tower**: No trip to Paris is complete without a visit to this iconic structure. You can take an elevator ride to the top for stunning views of the city.
  
2. **Louvre Museum**: Home to thousands of works of art, including the Mona Lisa and the Venus de Milo, the Louvre is a must-visit for art lovers.

3. **Notre-Dame Cathedral**: Despite the ongoing restoration, the façade and surrounding areas are still worth a visit. Explore the nearby Île de la Cité and Sainte-Chapelle with its stunning stained glass.

4. **Arc de Triomphe and Champs-Élysées**: Climb to the top of the Arc for panoramic views, and stroll down the famous avenue for shopping and dining.

5. **Montmartre and Sacré-Cœur Basilica**: Explore the charming streets of Montmartre, visit the famous basilica, and enjoy breathtaking views of the city from i

In [490]:
# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://host.docker.internal:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
model = 'Qwen/QwQ-32B-AWQ'

In [491]:
# Round 1
messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
# For granite, add: `extra_body={"chat_template_kwargs": {"thinking": True}}`
response = client.chat.completions.create(model=model, messages=messages)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print("reasoning_content:", reasoning_content)
print("content:", content)

reasoning_content: Okay, so I need to figure out whether 9.11 is greater than 9.8 or not. Hmm, let's see. Both numbers start with 9, so the whole number part is the same. That means I have to look at the decimal parts to compare them. 

First, I remember that when comparing decimals, you start from the first digit after the decimal point. So, let's break them down. 

For 9.11, the digits after the decimal are 1 and 1. So, the tenths place is 1 and the hundredths place is 1. 

For 9.8, the digits after the decimal are 8 and then... well, there's nothing after that. So, it's just 8 in the tenths place and 0 in the hundredths place, right? Because 9.8 is the same as 9.80. 

So, comparing the tenths place first: 9.11 has 1 in the tenths, and 9.8 has 8 in the tenths. Since 8 is greater than 1, that would mean that 9.8 is actually larger than 9.11. Wait, but hold on, that seems counterintuitive because 11 is more than 8, but maybe I'm mixing up the places. 

Let me think again. Tenths are th

In [ ]:
# Define a set of prompts with varying levels of specificity for time extraction
prompts = {
    "basic": "Extract clinical 'PROBLEM', 'TEST', 'TREATMENT' from the text and estimate the time of each event happened.",
    "structured": "For each clinical event (PROBLEM, TEST, TREATMENT) in the text, provide: 1) The event type, 2) Description, 3) Exact or estimated date/time it occurred, and 4) Your confidence level (high/medium/low).",
    "temporal_focus": "Focus specifically on the temporal information in the text. For each clinical event, determine when it happened relative to other events and to the document creation time. Provide dates when possible, otherwise use relative terms (e.g., '2 days before admission').",
    "json_format": "Extract the temporal information of clinical events as a JSON object. Each event should have: event_type (PROBLEM/TEST/TREATMENT), description, timestamp (exact or estimated), and temporal_indicators (words/phrases that helped you determine the time)."
}

def test_temporal_inference(text, prompt_type="basic"):
    chat_response = client.chat.completions.create(
        model = model,
        messages=[
            {"role": "system", "content": "You are a medical assistant with expertise in understanding clinical timelines and temporal relationships between medical events."},
            {"role": "user", "content": prompts[prompt_type] + "\n\nCLINICAL TEXT: " + text},
        ]
    )
    return chat_response.choices[0].message.content

# Test the basic extraction
if 'text' in locals():
    print("Testing basic temporal inference:")
    print(test_temporal_inference(text, "basic"))

Chat response: Okay, let's tackle this query. The user wants me to extract clinical problems, tests, and treatments from the provided medical text and estimate the timing of each event. The admission and discharge dates are given as 09/29/1993 to 10/04/1993. 

First, I need to read through the entire text carefully. The HISTORY OF PRESENT ILLNESS section mentions the patient has left upper quadrant pain, nausea, vomiting, which have been ongoing, getting worse over the past month. Also, there's a mention of HIV positive since 1991. The HOSPITAL COURSE describes what happened during the hospital stay, including procedures and treatments.

Starting with PROBLEMs. The main issues here are the left upper quadrant pain, nausea/vomiting, HIV infection, weight loss, and the abscesses in her legs. The abscess in the left lower extremity from 1991 and new ones in the left calf and right quadriceps. Also, failure to thrive and weight loss are noted. These should be listed as problems with their 

In [450]:
!pwd
!ls

/home/jovyan


work


In [457]:
def data_load(src_folder: str):
    # Get all .xml.label.txt files in the source folder
    label_files = [f for f in os.listdir(src_folder) if f.endswith('.xml.label.txt')]

    # Dictionary to store the organized data by file ID
    organized_data = {}

    # Process each label file
    for filename in label_files:
        # Extract the file ID (everything before .xml)
        file_id = filename.split('.xml')[0]
        
        # Initialize the dictionary for this ID
        organized_data[file_id] = {}

        # Load label text file
        label_path = os.path.join(src_folder, filename)
        with open(label_path, "r") as f:
            organized_data[file_id]['label_text'] = f.read()

        # Load corresponding starttime JSON file
        starttime_filename = f"{file_id}.xml.starttime.json"
        starttime_path = os.path.join(src_folder, starttime_filename)
        if os.path.exists(starttime_path):
            with open(starttime_path, "r") as f:
                organized_data[file_id]['starttime'] = json.load(f)
        else:
            organized_data[file_id]['starttime'] = None

        # Load corresponding interval_paths JSON file
        interval_paths_filename = f"{file_id}.xml.interval_paths.json"
        interval_paths_path = os.path.join(src_folder, interval_paths_filename)
        if os.path.exists(interval_paths_path):
            with open(interval_paths_path, "r") as f:
                organized_data[file_id]['interval_paths'] = json.load(f)
        else:
            organized_data[file_id]['interval_paths'] = None

    print(f"Loaded data for {len(organized_data)} IDs.")
    return organized_data

In [459]:
train_data = data_load('/home/jovyan/work/Temporal_relation/data/timeline_training/')
test_data = data_load('/home/jovyan/work/Temporal_relation/data/timeline_test/')

Loaded data for 190 IDs.
Loaded data for 120 IDs.


In [461]:
train_data.keys()

dict_keys(['751', '201', '98', '373', '332', '701', '512', '92', '42', '237', '546', '722', '307', '807', '532', '87', '471', '433', '388', '626', '346', '272', '336', '11', '193', '271', '316', '502', '777', '567', '417', '472', '291', '2', '122', '413', '572', '582', '331', '313', '393', '72', '321', '23', '636', '167', '786', '481', '362', '382', '591', '423', '736', '81', '166', '707', '422', '47', '38', '301', '492', '511', '656', '6', '51', '631', '647', '717', '186', '622', '787', '711', '411', '126', '267', '177', '216', '121', '1', '482', '367', '212', '473', '387', '188', '682', '308', '191', '521', '18', '721', '173', '797', '576', '637', '462', '641', '386', '357', '273', '318', '692', '577', '168', '68', '163', '541', '43', '757', '213', '203', '343', '311', '497', '337', '426', '348', '28', '252', '776', '463', '197', '153', '26', '596', '396', '376', '3', '496', '522', '557', '408', '491', '242', '756', '151', '236', '123', '427', '747', '152', '432', '366', '36', '458',

In [477]:
text = train_data['751']['label_text']
start_time = train_data['751']['starttime']

In [474]:
# start_time

In [479]:
def evaluate_temporal_inference(text, events, prompt_type="temporal_focus"):
    """Evaluate the model's ability to infer time for specific events"""
    results = []
    
    for event in events:
        eid = event['node_id']
        ground_truth_range = event['start_range']
        
        # Create a targeted prompt for this specific event
        event_prompt = f"Based on the clinical text, when exactly did the event marked with <{eid}> and </{eid}> occur? "
        event_prompt += "Provide the most specific date/time possible, whether exact or relative to other events. "
        event_prompt += "If you're uncertain, explain your level of confidence and the temporal clues you used."
        
        chat_response = client.chat.completions.create(
            model = model,
            messages=[
                {"role": "system", "content": "You are a medical assistant with expertise in understanding clinical timelines and temporal relationships between medical events."},
                {"role": "user", "content": event_prompt + "\n\nCLINICAL TEXT: " + text},
            ]
        )
        
        result = {
            "event_id": eid,
            "ground_truth": ground_truth_range,
            "llm_inference": chat_response.choices[0].message.content
        }
        results.append(result)
        
        print(f"\nEvent ID: {eid}")
        print(f"Ground truth time range: {ground_truth_range}")
        print(f"LLM inference: {chat_response.choices[0].message.content}\n")
        print("-" * 80)
    
    return results

In [ ]:
def compare_prompt_strategies(text, events, prompt_types=["basic", "structured", "temporal_focus", "json_format"]):
    """Compare different prompting strategies for temporal inference"""
    results = {}
    
    # Select a representative sample event
    if events and len(events) > 0:
        sample_event = events[0]
        eid = sample_event['node_id']
        
        print(f"Comparing prompt strategies for event ID: {eid}\n")
        
        for prompt_type in prompt_types:
            print(f"\n--- Using {prompt_type} prompt ---")
            
            event_text = text.replace(f"<{eid}>", f"**EVENT START**").replace(f"</{eid}>", f"**EVENT END**")
            
            response = test_temporal_inference(event_text, prompt_type)
            results[prompt_type] = response
            
            print(f"Response:\n{response}")
            print("-" * 80)
            
    return results

In [489]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import VLLMOpenAI


from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.output_parsers import StrOutputParser

def evaluate_temporal_inference(text, events):
    """Evaluate the model's ability to infer time for specific events"""
    results = []
    
    # Define system message once
    system_message = "You are a medical assistant with expertise in understanding clinical timelines and temporal relationships between medical events."
    
    for event in events:
        eid = event['node_id']
        ground_truth_range = event['start_range']
        
        # Create user message with clinical text
        event_prompt = f"Based on the clinical text, when exactly did the event marked with <{eid}> and </{eid}> occur? "
        event_prompt += "Provide the most specific date/time possible, whether exact or relative to other events. "
        event_prompt += "If you're uncertain, explain your level of confidence and the temporal clues you used."
        user_content = f"{event_prompt}\n\nCLINICAL TEXT: {text}"
        
        # Define messages in chat format
        messages = [
            SystemMessage(content=system_message),
            HumanMessage(content=user_content)
        ]
        
        # Create proper chat prompt template
        chat_prompt = ChatPromptTemplate.from_messages(messages)
        
        # Create chain with chat format
        chain = chat_prompt | llm | StrOutputParser()
        
        # Get response
        langchain_response = chain.invoke({})
        print(f"Langchain LLM inference: {langchain_response}\n")
        
        # Also get raw client response for comparison
        chat_response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_content},
            ]
        )
        raw_response = chat_response.choices[0].message.content
        
        # Store results
        result = {
            "event_id": eid,
            "ground_truth": ground_truth_range,
            "langchain_inference": langchain_response,
            "raw_inference": raw_response
        }
        results.append(result)
        
        # Print results
        print(f"\nEvent ID: {eid}")
        print(f"Ground truth time range: {ground_truth_range}")
        print(f"Langchain inference: {langchain_response}")
        print(f"Raw client inference: {raw_response}\n")
        print("-" * 80)
    
    return results


model_name = 'Qwen/QwQ-32B-AWQ'
# Earlier in your code, modify the llm initialization
llm = VLLMOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    model_name=model_name,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

def evaluate_temporal_inference(text, events):
    """Evaluate the model's ability to infer time for specific events"""
    results = []
    
    # Define system message just once (same as in your raw implementation)
    system_message = "You are a medical assistant with expertise in understanding clinical timelines and temporal relationships between medical events."
    
    for event in events:
        eid = event['node_id']
        ground_truth_range = event['start_range']
        
        # Create a targeted prompt for this specific event
        event_prompt = f"Based on the clinical text, when exactly did the event marked with <{eid}> and </{eid}> occur? "
        event_prompt += "Provide the most specific date/time possible, whether exact or relative to other events. "
        event_prompt += "If you're uncertain, explain your level of confidence and the temporal clues you used."
        
        user_content = f"{event_prompt}\n\nCLINICAL TEXT: {text}"
        
        # Create a chat prompt template with system and user messages
        chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=system_message),
            HumanMessage(content=user_content)
        ])
        
        # Build the chain with the prompt template
        chain = chat_prompt | llm | StrOutputParser()
        
        # Get langchain response
        langchain_response = chain.invoke({})
        print(f"Langchain LLM inference: {langchain_response}\n")
        print("*" * 80)

        # Also get raw client response for comparison
        chat_response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_content},
            ]
        )
        raw_response = chat_response.choices[0].message.content

        result = {
            "event_id": eid,
            "ground_truth": ground_truth_range,
            "langchain_inference": langchain_response,
            "raw_inference": raw_response
        }
        results.append(result)
        
        print(f"\nEvent ID: {eid}")
        print(f"Ground truth time range: {ground_truth_range}")
        print(f"Langchain inference: {langchain_response}")
        print(f"Raw client inference: {raw_response}\n")
        print("-" * 80)
    
    return results

ImportError: cannot import name 'StrOutputParser' from 'langchain.output_parsers' (/opt/conda/lib/python3.11/site-packages/langchain/output_parsers/__init__.py)

In [488]:
# evaluate_temporal_inference(text, start_time)

In [494]:
def preprocess_text(text, event_id):
    """
    Removes all event tags <E#> and </E#> from the text, except for the specified event tag.
    
    Args:
        text (str): The input text containing event tags
        event_id (str): The event ID to keep (e.g., "E2", "E5", etc.)
        
    Returns:
        str: The text with all event tags removed except for the specified event
    """
    import re
    
    # Strip the 'E' prefix if it exists to get just the number
    if event_id.startswith('E'):
        event_num = event_id[1:]
    else:
        event_num = event_id
        event_id = 'E' + event_id
    
    # Create the pattern to match all <E#> and </E#> tags except the specified one
    pattern = r'<E(?!{0}>)(\d+)>|</E(?!{0}>)(\d+)>'.format(event_num)
    
    # Remove all matches of the pattern
    processed_text = re.sub(pattern, '', text)
    
    return processed_text

In [496]:
# This will keep <E5> and </E5> tags and remove all others
text = "The patient is a 28-year-old woman who is <E2>HIV positive</E2> for two years. " \
       "She presented with <E4>left upper quadrant pain</E4> as well as <E5>nausea</E5> and <E6>vomiting</E6>."
       
processed_text = preprocess_text(text, "E2")
print(processed_text)
# Output: "The patient is a 28-year-old woman who is HIV positive for two years. " \
#        "She presented with left upper quadrant pain as well as <E5>nausea</E5> and vomiting."


The patient is a 28-year-old woman who is <E2>HIV positive</E2> for two years. She presented with left upper quadrant pain as well as nausea and vomiting.


In [2]:
list1 =['316', '197', '468', '163', '682', '18', '272', '521', '201', '511', '8', '411', '386', '582', '462', '152', '757', '236', '357', '237', '122', '23', '458', '311', '313', '26', '3', '167', '577', '11', '166', '173', '497', '526', '321', '502', '216', '93', '203', '393', '591', '367', '491', '482', '676', '636', '51', '331', '567', '182']
list2 = ['751', '201', '98', '373', '332', '701', '512', '92', '42', '237', '546', '722', '307', '807', '532', '87', '471', '433', '388', '626', '346', '272', '336', '11', '193', '271', '316', '502', '777', '567', '417', '472', '291', '2', '122', '413', '572', '582', '331', '313', '393', '72', '321', '23', '636', '167', '786', '481', '362', '382']
list3 = ['316', '197', '468', '163', '682', '18', '272', '521', '201', '511', '8', '411', '386', '582', '462', '152', '757', '236', '357', '237', '122', '23', '458', '311', '313', '26', '3', '167', '577', '11', '166', '173', '497', '526', '321', '502', '216', '93', '203', '393', '591', '367', '491', '482', '676', '636', '51']

In [8]:
set(list1) == set(list2)


False

In [9]:
set(list3) == set(list2)

False

In [10]:
set(list1) == set(list3)


False

In [12]:
# ['1.xml.notime.label.txt', '107.xml.notime.label.txt', '11.xml.notime.label.txt', '116.xml.notime.label.txt', '121.xml.notime.label.txt', '122.xml.notime.label.txt', '123.xml.notime.label.txt', '126.xml.notime.label.txt', '141.xml.notime.label.txt', '143.xml.notime.label.txt', '151.xml.notime.label.txt', '152.xml.notime.label.txt', '153.xml.notime.label.txt', '156.xml.notime.label.txt', '16.xml.notime.label.txt', '162.xml.notime.label.txt', '163.xml.notime.label.txt', '166.xml.notime.label.txt', '167.xml.notime.label.txt', '168.xml.notime.label.txt', '172.xml.notime.label.txt', '173.xml.notime.label.txt', '177.xml.notime.label.txt', '178.xml.notime.label.txt', '18.xml.notime.label.txt', '182.xml.notime.label.txt', '186.xml.notime.label.txt', '188.xml.notime.label.txt', '191.xml.notime.label.txt', '192.xml.notime.label.txt', '193.xml.notime.label.txt', '197.xml.notime.label.txt', '2.xml.notime.label.txt', '201.xml.notime.label.txt', '203.xml.notime.label.txt', '212.xml.notime.label.txt', '213.xml.notime.label.txt', '216.xml.notime.label.txt', '218.xml.notime.label.txt', '23.xml.notime.label.txt', '236.xml.notime.label.txt', '237.xml.notime.label.txt', '242.xml.notime.label.txt', '247.xml.notime.label.txt', '248.xml.notime.label.txt', '252.xml.notime.label.txt', '256.xml.notime.label.txt', '26.xml.notime.label.txt', '267.xml.notime.label.txt', '271.xml.notime.label.txt', '272.xml.notime.label.txt', '273.xml.notime.label.txt', '28.xml.notime.label.txt', '291.xml.notime.label.txt', '3.xml.notime.label.txt', '301.xml.notime.label.txt', '302.xml.notime.label.txt', '307.xml.notime.label.txt', '308.xml.notime.label.txt', '311.xml.notime.label.txt', '313.xml.notime.label.txt', '316.xml.notime.label.txt', '318.xml.notime.label.txt', '321.xml.notime.label.txt', '331.xml.notime.label.txt', '332.xml.notime.label.txt', '336.xml.notime.label.txt', '337.xml.notime.label.txt', '343.xml.notime.label.txt', '346.xml.notime.label.txt', '348.xml.notime.label.txt', '351.xml.notime.label.txt', '352.xml.notime.label.txt', '353.xml.notime.label.txt', '356.xml.notime.label.txt', '357.xml.notime.label.txt', '36.xml.notime.label.txt', '362.xml.notime.label.txt', '366.xml.notime.label.txt', '367.xml.notime.label.txt', '373.xml.notime.label.txt', '376.xml.notime.label.txt', '38.xml.notime.label.txt', '382.xml.notime.label.txt', '386.xml.notime.label.txt', '387.xml.notime.label.txt', '388.xml.notime.label.txt', '393.xml.notime.label.txt', '396.xml.notime.label.txt', '407.xml.notime.label.txt', '408.xml.notime.label.txt', '411.xml.notime.label.txt', '413.xml.notime.label.txt', '417.xml.notime.label.txt', '42.xml.notime.label.txt', '422.xml.notime.label.txt', '423.xml.notime.label.txt', '426.xml.notime.label.txt', '427.xml.notime.label.txt', '43.xml.notime.label.txt', '432.xml.notime.label.txt', '433.xml.notime.label.txt', '437.xml.notime.label.txt', '438.xml.notime.label.txt', '452.xml.notime.label.txt', '458.xml.notime.label.txt', '462.xml.notime.label.txt', '463.xml.notime.label.txt', '468.xml.notime.label.txt', '47.xml.notime.label.txt', '471.xml.notime.label.txt', '472.xml.notime.label.txt', '473.xml.notime.label.txt', '481.xml.notime.label.txt', '482.xml.notime.label.txt', '491.xml.notime.label.txt', '492.xml.notime.label.txt', '496.xml.notime.label.txt', '497.xml.notime.label.txt', '502.xml.notime.label.txt', '51.xml.notime.label.txt', '511.xml.notime.label.txt', '512.xml.notime.label.txt', '517.xml.notime.label.txt', '521.xml.notime.label.txt', '522.xml.notime.label.txt', '526.xml.notime.label.txt', '532.xml.notime.label.txt', '541.xml.notime.label.txt', '546.xml.notime.label.txt', '547.xml.notime.label.txt', '557.xml.notime.label.txt', '567.xml.notime.label.txt', '571.xml.notime.label.txt', '572.xml.notime.label.txt', '576.xml.notime.label.txt', '577.xml.notime.label.txt', '582.xml.notime.label.txt', '587.xml.notime.label.txt', '591.xml.notime.label.txt', '596.xml.notime.label.txt', '6.xml.notime.label.txt', '602.xml.notime.label.txt', '611.xml.notime.label.txt', '612.xml.notime.label.txt', '622.xml.notime.label.txt', '626.xml.notime.label.txt', '631.xml.notime.label.txt', '636.xml.notime.label.txt', '637.xml.notime.label.txt', '641.xml.notime.label.txt', '642.xml.notime.label.txt', '647.xml.notime.label.txt', '656.xml.notime.label.txt', '666.xml.notime.label.txt', '676.xml.notime.label.txt', '68.xml.notime.label.txt', '681.xml.notime.label.txt', '682.xml.notime.label.txt', '692.xml.notime.label.txt', '697.xml.notime.label.txt', '701.xml.notime.label.txt', '707.xml.notime.label.txt', '711.xml.notime.label.txt', '717.xml.notime.label.txt', '72.xml.notime.label.txt', '721.xml.notime.label.txt', '722.xml.notime.label.txt', '726.xml.notime.label.txt', '736.xml.notime.label.txt', '747.xml.notime.label.txt', '751.xml.notime.label.txt', '756.xml.notime.label.txt', '757.xml.notime.label.txt', '776.xml.notime.label.txt', '777.xml.notime.label.txt', '786.xml.notime.label.txt', '787.xml.notime.label.txt', '791.xml.notime.label.txt', '797.xml.notime.label.txt', '8.xml.notime.label.txt', '801.xml.notime.label.txt', '807.xml.notime.label.txt', '81.xml.notime.label.txt', '86.xml.notime.label.txt', '87.xml.notime.label.txt', '92.xml.notime.label.txt', '93.xml.notime.label.txt', '96.xml.notime.label.txt', '98.xml.notime.label.txt']

In [ ]:
# ['1.xml.label.txt', '107.xml.label.txt', '11.xml.label.txt', '116.xml.label.txt', '121.xml.label.txt', '122.xml.label.txt', '123.xml.label.txt', '126.xml.label.txt', '141.xml.label.txt', '143.xml.label.txt', '151.xml.label.txt', '152.xml.label.txt', '153.xml.label.txt', '156.xml.label.txt', '16.xml.label.txt', '162.xml.label.txt', '163.xml.label.txt', '166.xml.label.txt', '167.xml.label.txt', '168.xml.label.txt', '172.xml.label.txt', '173.xml.label.txt', '177.xml.label.txt', '178.xml.label.txt', '18.xml.label.txt', '182.xml.label.txt', '186.xml.label.txt', '188.xml.label.txt', '191.xml.label.txt', '192.xml.label.txt', '193.xml.label.txt', '197.xml.label.txt', '2.xml.label.txt', '201.xml.label.txt', '203.xml.label.txt', '212.xml.label.txt', '213.xml.label.txt', '216.xml.label.txt', '218.xml.label.txt', '23.xml.label.txt', '236.xml.label.txt', '237.xml.label.txt', '242.xml.label.txt', '247.xml.label.txt', '248.xml.label.txt', '252.xml.label.txt', '256.xml.label.txt', '26.xml.label.txt', '267.xml.label.txt', '271.xml.label.txt', '272.xml.label.txt', '273.xml.label.txt', '28.xml.label.txt', '291.xml.label.txt', '3.xml.label.txt', '301.xml.label.txt', '302.xml.label.txt', '307.xml.label.txt', '308.xml.label.txt', '311.xml.label.txt', '313.xml.label.txt', '316.xml.label.txt', '318.xml.label.txt', '321.xml.label.txt', '331.xml.label.txt', '332.xml.label.txt', '336.xml.label.txt', '337.xml.label.txt', '343.xml.label.txt', '346.xml.label.txt', '348.xml.label.txt', '351.xml.label.txt', '352.xml.label.txt', '353.xml.label.txt', '356.xml.label.txt', '357.xml.label.txt', '36.xml.label.txt', '362.xml.label.txt', '366.xml.label.txt', '367.xml.label.txt', '373.xml.label.txt', '376.xml.label.txt', '38.xml.label.txt', '382.xml.label.txt', '386.xml.label.txt', '387.xml.label.txt', '388.xml.label.txt', '393.xml.label.txt', '396.xml.label.txt', '407.xml.label.txt', '408.xml.label.txt', '411.xml.label.txt', '413.xml.label.txt', '417.xml.label.txt', '42.xml.label.txt', '422.xml.label.txt', '423.xml.label.txt', '426.xml.label.txt', '427.xml.label.txt', '43.xml.label.txt', '432.xml.label.txt', '433.xml.label.txt', '437.xml.label.txt', '438.xml.label.txt', '452.xml.label.txt', '458.xml.label.txt', '462.xml.label.txt', '463.xml.label.txt', '468.xml.label.txt', '47.xml.label.txt', '471.xml.label.txt', '472.xml.label.txt', '473.xml.label.txt', '481.xml.label.txt', '482.xml.label.txt', '491.xml.label.txt', '492.xml.label.txt', '496.xml.label.txt', '497.xml.label.txt', '502.xml.label.txt', '51.xml.label.txt', '511.xml.label.txt', '512.xml.label.txt', '517.xml.label.txt', '521.xml.label.txt', '522.xml.label.txt', '526.xml.label.txt', '532.xml.label.txt', '541.xml.label.txt', '546.xml.label.txt', '547.xml.label.txt', '557.xml.label.txt', '567.xml.label.txt', '571.xml.label.txt', '572.xml.label.txt', '576.xml.label.txt', '577.xml.label.txt', '582.xml.label.txt', '587.xml.label.txt', '591.xml.label.txt', '596.xml.label.txt', '6.xml.label.txt', '602.xml.label.txt', '611.xml.label.txt', '612.xml.label.txt', '622.xml.label.txt', '626.xml.label.txt', '631.xml.label.txt', '636.xml.label.txt', '637.xml.label.txt', '641.xml.label.txt', '642.xml.label.txt', '647.xml.label.txt', '656.xml.label.txt', '666.xml.label.txt', '676.xml.label.txt', '68.xml.label.txt', '681.xml.label.txt', '682.xml.label.txt', '692.xml.label.txt', '697.xml.label.txt', '701.xml.label.txt', '707.xml.label.txt', '711.xml.label.txt', '717.xml.label.txt', '72.xml.label.txt', '721.xml.label.txt', '722.xml.label.txt', '726.xml.label.txt', '736.xml.label.txt', '747.xml.label.txt', '751.xml.label.txt', '756.xml.label.txt', '757.xml.label.txt', '776.xml.label.txt', '777.xml.label.txt', '786.xml.label.txt', '787.xml.label.txt', '791.xml.label.txt', '797.xml.label.txt', '8.xml.label.txt', '801.xml.label.txt', '807.xml.label.txt', '81.xml.label.txt', '86.xml.label.txt', '87.xml.label.txt', '92.xml.label.txt', '93.xml.label.txt', '96.xml.label.txt', '98.xml.label.txt']

# ['1', '107', '11', '116', '121', '122', '123', '126', '141', '143']
# ['1', '107', '11', '116', '121', '122', '123', '126', '141', '143']
